In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


In [ ]:
# Define constants
IMAGE_SIZE = (250, 250)
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.0001


In [ ]:
# Function to load images from a directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, IMAGE_SIZE)
            images.append(img)
    return images

In [ ]:
# Load images from folders
healthy_images = load_images_from_folder('/content/drive/MyDrive/FYP/Datasets/Plantsdatasets/healthyimages')
diseased_images = load_images_from_folder('/content/drive/MyDrive/FYP/Datasets/Plantsdatasets/images')


In [ ]:
# Create labels
healthy_labels = [0] * len(healthy_images)
diseased_labels = [1] * len(diseased_images)

# Combine images and labels
X = np.array(healthy_images + diseased_images)
y = np.array(healthy_labels + diseased_labels)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data normalization
X_train = X_train / 255
X_test = X_test / 255


In [ ]:

# Define the model
model = Sequential([
    Flatten(input_shape=IMAGE_SIZE + (3,)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Define callbacks for early stopping and model checkpoint
callbacks = [
    EarlyStopping(patience=3, monitor='val_loss'),
    ModelCheckpoint(filepath='best_Classification_model.h5', monitor='val_loss', save_best_only=True)
]


In [ ]:
# Train the model with callbacks
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
                    validation_split=0.1, callbacks=callbacks)

In [ ]:
# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Testing Accuracy: {accuracy*100:.2f}")

Trainaccuracy = model.evaluate(X_train, y_train)[1]
print(f"Training Accuracy: {Trainaccuracy*100:.2f}")

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Load the trained model
model = load_model('/content/best_Classification_model.h5')  # Path to your trained model


In [ ]:
# Function to preprocess image
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (250, 250))
    img = img / 255.0  # Normalize the image
    return img

# Function to make prediction
def predict_image(image_path):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = model.predict(img)[0][0]  # Get the prediction probability
    if prediction >= 0.5:
        return "Diseased"
    else:
        return "Healthy"

In [ ]:
# Path to the image you want to predict
image_path = '/content/drive/MyDrive/FYP/Datasets/Plantsdatasets/healthy/00002.jpg'

# Predict the image
prediction = predict_image(image_path)
print("Prediction:", prediction)

# Load and plot the image
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
plt.imshow(img)
plt.axis('off')
plt.title(f'Prediction: {prediction}')
plt.show()

In [ ]:
import shutil

# Define source and destination paths
source_path = '/content/best_Classification_model.h5'
destination_path = '/content/drive/MyDrive/FYP/Trained Models/best_Classification_model.h5'

# Move the file to the destination
shutil.move(source_path, destination_path)

print(f"File moved from {source_path} to {destination_path}")
